In [3]:
from database.adatabase import ADatabase
from extractor.alp_client_extractor import ALPClientExtractor
from extractor.tiingo_extractor import TiingoExtractor
from processor.processor import Processor as processor
from datetime import datetime, timedelta
from tqdm import tqdm 
from dotenv import load_dotenv
load_dotenv()
import os
import pandas as pd
from time import sleep
import matplotlib.pyplot as plt



In [9]:
extractor = ALPClientExtractor(os.getenv("APCAKEY"),os.getenv("APCASECRET"))
tickers = ["AAVE", "AVAX", "BAT", "BCH", "BTC", "CRV", "DOGE", "DOT", "ETH", "GRT", "LINK", "LTC", "MKR", "SHIB", "SUSHI","USDT","USDC", "UNI", "XTZ", "YFI"]
tickers = [x +"/USD" for x in tickers]
prices = []
for ticker in tqdm(tickers):
    price_df = []
    for i in range(1,4):
        start = datetime.now() - timedelta(days=7*i)
        end = datetime.now() - timedelta(days=7*(i-1))
        response = extractor.crypto(ticker,start,end)
        price = pd.DataFrame(response["bars"][ticker]).rename(columns={"c":"adjclose","t":"date"})[["date","adjclose"]]
        price["date"] = pd.to_datetime(price["date"])
        price = price.sort_values("date")
        price_df.append(price)
    price = pd.concat(price_df).sort_values("date")
    price["rolling_return"] = ((price["adjclose"].rolling(30).mean() - price["adjclose"]) / price["adjclose"])
    price["return"] = price["adjclose"].pct_change(5)
    price["avg_return"] = price["return"].rolling(30).mean()
    price["coev"] = (price["adjclose"].rolling(30).mean() / price["adjclose"].rolling(30).std())
    price["ticker"]= ticker
    prices.append(price)

prices = pd.concat(prices).sort_values("date").dropna()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:13<00:00,  1.46it/s]


In [10]:
prices

,date,adjclose,rolling_return,return,avg_return,coev,ticker
34,2024-09-05 00:34:00+00:00,135.832500,0.004030,-0.004784,0.001551,231.727155,AAVE/USD
35,2024-09-05 00:35:00+00:00,135.497000,0.006676,-0.005818,0.001411,252.418808,AAVE/USD
34,2024-09-05 00:36:00+00:00,1.000150,-0.000101,0.000250,0.000024,9409.364608,USDC/USD
34,2024-09-05 00:36:00+00:00,0.000014,0.002090,-0.000737,-0.000255,494.103700,SHIB/USD
36,2024-09-05 00:36:00+00:00,135.644000,0.005627,-0.006435,0.001067,256.854820,AAVE/USD
...,...,...,...,...,...,...,...
4665,2024-09-26 00:00:00+00:00,0.189289,-0.000510,0.001953,-0.000341,609.323583,BAT/USD
6822,2024-09-26 00:00:00+00:00,27.275000,-0.000583,0.003276,-0.000448,459.540191,AVAX/USD
7105,2024-09-26 00:00:00+00:00,165.130500,-0.000416,0.001398,-0.000012,504.666859,AAVE/USD
9490,2024-09-26 00:00:00+00:00,0.000015,-0.004171,0.000984,0.000244,441.711172,SHIB/USD


In [11]:
metrics = [
    "rolling_return",
    "return",
    "avg_return"
    ,"coev"
]

In [23]:
date_range = list(prices["date"].unique())

cash = 100
base_position = {"date":date_range[0],"ticker":"","quantity":0,"adjclose":0,"buy_price":0,"buy_date":date_range[0]}
positions = []
analysis = []
for metric in tqdm(metrics):
    for ascending in [True,False]:
        position = base_position.copy()
        for date in date_range:
            try:
                position = position.copy()
                position["date"] = date
                today = prices[prices["date"]==date]
                opportunity = today.sort_values(metric,ascending=ascending).iloc[0]
                if today.index.size >= len(tickers):
                    if position["ticker"] != "":
                        position["adjclose"] = today[today["ticker"]==position["ticker"]]["adjclose"].iloc[0].item()
                        if (date - position["buy_date"]).days > float(5/(24*60)):
                            cash = position["adjclose"] * position["quantity"]
                            position = base_position.copy()
                            position["date"] = date
                            position["ticker"] = opportunity["ticker"]
                            position["quantity"] = cash * 0.9975 / opportunity["adjclose"] 
                            position["adjclose"] = opportunity["adjclose"]
                            position["buy_price"] = opportunity["adjclose"]
                            position["buy_date"] = date
                    else:
                            position["date"] = date
                            position["ticker"] = opportunity["ticker"]
                            position["quantity"] = cash * 0.9975 / opportunity["adjclose"] 
                            position["adjclose"] = opportunity["adjclose"]
                            position["buy_price"] = opportunity["adjclose"]
                            position["buy_date"] = date
                positions.append(position.copy())
            except Exception as e:
                print(str(e))
        position["metric"] = metric
        position["ascending"] = ascending
        position["pv"] = position["adjclose"] * position["quantity"]
        analysis.append(position)

 50%|█████████████████████████████████████████████████████████████████████████████████████████████████████▌                                                                                                     | 2/4 [01:56<01:56, 58.34s/it]

single positional indexer is out-of-bounds


 75%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                  | 3/4 [02:55<00:58, 58.32s/it]

single positional indexer is out-of-bounds


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [03:53<00:00, 58.28s/it]


In [24]:
a = pd.DataFrame(analysis)
a.sort_values("pv",ascending=False)

,date,ticker,quantity,adjclose,buy_price,buy_date,metric,ascending,pv
6,2024-09-26 00:00:00+00:00,CRV/USD,990.711895,0.306720,0.308050,2024-09-25 15:52:00+00:00,coev,True,303.871152
7,2024-09-26 00:00:00+00:00,USDT/USD,295.738265,0.999863,0.999730,2024-09-25 15:52:00+00:00,coev,False,295.697749
5,2024-09-26 00:00:00+00:00,LTC/USD,3.472163,67.319150,67.471350,2024-09-25 15:52:00+00:00,avg_return,False,233.743091
4,2024-09-26 00:00:00+00:00,CRV/USD,618.294595,0.306720,0.308050,2024-09-25 15:52:00+00:00,avg_return,True,189.643318
3,2024-09-26 00:00:00+00:00,CRV/USD,563.099531,0.306720,0.308050,2024-09-25 15:52:00+00:00,return,False,172.713888
2,2024-09-26 00:00:00+00:00,BAT/USD,753.001861,0.190115,0.192250,2024-09-25 15:52:00+00:00,return,True,143.156949
1,2024-09-26 00:00:00+00:00,BAT/USD,679.258425,0.190115,0.192250,2024-09-25 15:52:00+00:00,rolling_return,False,129.137215
0,2024-09-26 00:00:00+00:00,USDC/USD,120.768545,0.999822,0.999924,2024-09-25 15:52:00+00:00,rolling_return,True,120.747109


In [25]:
price

,date,adjclose,rolling_return,return,avg_return,coev,ticker
0,2024-09-05 00:01:00+00:00,4952.9500,NaN,NaN,NaN,NaN,YFI/USD
1,2024-09-05 00:02:00+00:00,4956.0250,NaN,NaN,NaN,NaN,YFI/USD
2,2024-09-05 00:08:00+00:00,4952.4750,NaN,NaN,NaN,NaN,YFI/USD
3,2024-09-05 00:11:00+00:00,4955.4420,NaN,NaN,NaN,NaN,YFI/USD
4,2024-09-05 00:13:00+00:00,4958.9700,NaN,NaN,NaN,NaN,YFI/USD
...,...,...,...,...,...,...,...
2691,2024-09-25 23:50:00+00:00,5101.3700,0.006477,-0.003564,-0.001334,284.430627,YFI/USD
2692,2024-09-25 23:53:00+00:00,5109.5500,0.004596,-0.000492,-0.001363,280.100418,YFI/USD
2693,2024-09-25 23:54:00+00:00,5115.7750,0.003132,0.002063,-0.001337,282.042323,YFI/USD
2694,2024-09-25 23:55:00+00:00,5118.7375,0.002353,0.004545,-0.001232,284.388387,YFI/USD
